# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

G. Guiglion  ->  G. Guiglion  |  ['G. Guiglion']
Arxiv has 42 new papers today
          1 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2402.06076


extracting tarball to tmp_2402.06076...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:120: LatexWarning: attempting recovering figure fig-Compare_wg11_wg10_24112021_reduce
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:125: LatexWarning: Recovered figure fig-Compare_wg11_wg10_24112021_reduce as tmp_2402.06076/./fig-Compare_wg11_wg10_24112021_reduce.png
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


Found 94 bibliographic references in tmp_2402.06076/worley_wg10homog.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2402.06076-b31b1b.svg)](https://arxiv.org/abs/arXiv:2402.06076) | **The Gaia-ESO Survey: The DR5 analysis of the medium-resolution GIRAFFE  and high-resolution UVES spectra of FGK-type stars**  |
|| C. C. Worley, et al. -- incl., <mark>G. Guiglion</mark> |
|*Appeared on*| *2024-02-12*|
|*Comments*| *31 pages, 19 figures*|
|**Abstract**| The Gaia-ESO Survey is an European Southern Observatory (ESO) public spectroscopic survey that targeted $10^5$ stars in the Milky Way covering the major populations of the disk, bulge and halo. The observations were made using FLAMES on the VLT obtaining both UVES high ($R\sim47,000$) and GIRAFFE medium ($R\sim20,000$) resolution spectra. The analysis of the Gaia-ESO spectra was the work of multiple analysis teams (nodes) within five working groups (WG). The homogenisation of the stellar parameters within WG11 (high resolution observations of FGK stars) and the homogenisation of the stellar parameters within WG10 (medium resolution observations of FGK stars) is described here. In both cases, the homogenisation was carried out using a bayesian Inference method developed specifically for the Gaia-ESO Survey by WG11. The WG10 homogenisation primarily used the cross-match of stars with WG11 as the reference set in both the stellar parameter and chemical abundance homogenisation. In this way the WG10 homogenised results have been placed directly onto the WG11 stellar parameter and chemical abundance scales. The reference set for the metal-poor end was sparse which limited the effectiveness of the homogenisation in that regime. For WG11, the total number of stars for which stellar parameters were derived was 6,231 with typical uncertainties for Teff, log g and [Fe/H] of 32~K, 0.05 and 0.05 respectively. One or more chemical abundances out of a possible 39 elements were derived for 6,188 of the stars. For WG10, the total number of stars for which stellar parameters were derived was 76,675 with typical uncertainties for Teff, log g and [Fe/H] of 64~K, 0.15 and 0.07 respectively. One or more chemical abundances out of a possible 30 elements were derived for 64,177 of the stars. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2402.06076.md
    + _build/html/tmp_2402.06076/./fig-HR15N_check_refsample_reduce.png
    + _build/html/tmp_2402.06076/./fig-HR10HR21_check_refsample_reduce.png
    + _build/html/tmp_2402.06076/./fig-WG10_HRD_Final_SNR_NN_Error_iDR6_reduce.png
    + _build/html/tmp_2402.06076/./fig-WG10Rec_Abundance_Error_24112021_reduce.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\teff}{T_\mathrm{eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\feh}{[Fe/H]}$</div>



<div id="title">

# The Gaia-ESO Survey: The DR5 analysis of the medium-resolution GIRAFFE and high-resolution UVES spectra of FGK-type stars$\thanks{Based on observations collected at the ESO telescopes under programme 188.B3002, the Gaia-ESO Public Spectroscopic Survey.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2402.06076-b31b1b.svg)](https://arxiv.org/abs/2402.06076)<mark>Appeared on: 2024-02-12</mark> -  _31 pages, 19 figures_

</div>
<div id="authors">

C. C. Worley, et al. -- incl., <mark>G. Guiglion</mark>

</div>
<div id="abstract">

**Abstract:** The Gaia-ESO Survey is an European Southern Observatory (ESO) public spectroscopic survey that targeted $10^5$ stars in the Milky Way covering the major populations of the disk, bulge and halo. The observations were made using FLAMES on the VLT obtaining both UVES high ( $R\sim47,000$ ) and GIRAFFE medium ( $R\sim20,000$ ) resolution spectra.  The analysis of the Gaia-ESO spectra was the work of multiple analysis teams (nodes) within five working groups (WG). The homogenisation of the stellar parameters within WG11 (high resolution observations of FGK stars) and the homogenisation of the stellar parameters within WG10 (medium resolution observations of FGK stars) is described here.  In both cases, the homogenisation was carried out using a bayesian Inference method developed specifically for the Gaia-ESO Survey by WG11. The method was also used for the chemical abundance homogenisation within WG11, however, the WG10 chemical abundance data set was too sparsely populated so basic corrections for each node analysis were employed for the homogenisation instead.  The WG10 homogenisation primarily used the cross-match of stars with WG11 as the reference set in both the stellar parameter and chemical abundance homogenisation. In this way the WG10 homogenised results have been placed directly onto the WG11 stellar parameter and chemical abundance scales. The reference set for the metal-poor end was sparse which limited the effectiveness of the homogenisation in that regime.  For WG11, the total number of stars for which stellar parameters were derived was 6,231 with typical uncertainties for $\teff$ , $\logg$ and $\feh$ of 32 K, 0.05 and 0.05 respectively. One or more chemical abundances out of a possible 39 elements were derived for 6,188 of the stars.  For WG10, the total number of stars for which stellar parameters were derived was 76,675 with typical uncertainties for $\teff$ , $\logg$ and $\feh$ of 64 K, 0.15 and 0.07 respectively. One or more chemical abundances out of a possible 30 elements were derived for 64,177 of the stars.

</div>

<div id="div_fig1">

<img src="tmp_2402.06076/./fig-HR15N_check_refsample_reduce.png" alt="Fig6.1" width="50%"/><img src="tmp_2402.06076/./fig-HR10HR21_check_refsample_reduce.png" alt="Fig6.2" width="50%"/>

**Figure 6. -** Reference sets for HR15N and HR10$|$HR21 setups. Top row: Kiel diagrams for full HR15N sample, final HR15N reference set and WG11 cross-match with HR15N. Second row: Same as for the top row but for $\feh$  versus $\teff$ . Third row: Kiel diagrams of the four main samples: benchmarks (yellow), OC (magenta), GCs (green), and MW (blue) within the HR15N reference set overlaid on the full reference set. Fourth row: Same as for the third row but for $\feh$  versus $\teff$ .  Fifth to eighth rows: Same but for HR10$|$HR21. (*fig:hr1021_refset*)

</div>
<div id="div_fig2">

<img src="tmp_2402.06076/./fig-WG10_HRD_Final_SNR_NN_Error_iDR6_reduce.png" alt="Fig10" width="100%"/>

**Figure 10. -** Characterisation of the final WG10 stellar parameters with histograms (top row), bins in Kiel diagrams (middle row) and bins in metallicity distribution (bottom row). Specific panel content are: a,f,k) S/N; b,g,l) Number of Nodes (NN); c,h,m) Error on $\teff$ ; d,i,n) Error on $\logg$ ; e,j,o) Error on $\feh$ (*fig:hrd_fin_qc*)

</div>
<div id="div_fig3">

<img src="tmp_2402.06076/./fig-WG10Rec_Abundance_Error_24112021_reduce.png" alt="Fig14" width="100%"/>

**Figure 14. -** Errors on WG10-recommended abundances (in absolute abundance) against S/N on a log scale. Sub-samples per NN are shown as specified in the top-left panel. (*fig:rec_abunerrs*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2402.06076"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

369  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
